# Recurrent Neural Networks

### A first Recurrent Neural Network layer

`SimpleRNN` processes batches of sequences, like all other Keras layers. This means that it takes inputs of shape `(batch_size, timesteps, input_features)`, rather than `(timesteps, input_features)`.


**As first thing, from Keras import `SimpleNN`.**

Create the first simple following network:
- A first `Embedding` layer of input dimension 10000 and output dimension 32;
- A `SimpleRNN` (Fully-connected RNN, where the output is to be fed back to input) with 32 hidden units;

** Build the Network as described above and check for a summary of it. **

** Add the `return_sequences` constructure argument. Check the documentation to better understand how it works.**

It is sometimes useful to stack several recurrent layers one after the other in order to increase the representational power of a network. In such a setup, you have to get all intermediate layers to return full sequences.

** Add 3 others intermediate layer of 32 hidden units each. Get a summary of the network.**

### Dataset and Data Preprocessing

Now let's try to use such a model on the IMDB movie review classification problem. First, we have to preprocess the data:

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)


** Check the dimension of train data and test data. **

Let's train a simple recurrent network using:
- an `Embedding` layer of input size equal to max_features and output equal to 32;
- a `SimpleRNN` layer;

** Build the network as described above. **

** Compile the model with an optimizer and a loss.**

** Fit the model, consider 10 epochs, some batch size and a validation split.**

Let's display the training and validation loss and accuracy:

** Fill in the #DO TOs to complete the plotting. **

In [ ]:
import matplotlib.pyplot as plt


acc = #TO DO: training accuracy
val_acc = #TO DO: validation accuracy
loss =  #TO DO: training loss
val_loss = #TO DO: validation loss

epochs = range(len(acc))

# Accuracy
plt.plot(epochs, #TO DO: training accuracy, 'bo', label='Training acc') 
plt.plot(epochs, #TO DO: validation accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Loss
plt.plot(epochs, #TO DO: traning loss, 'bo', label='Training loss') #TO DO
plt.plot(epochs, #TO DO: validation loss, 'b', label='Validation loss') #TO DO
plt.title('Training and validation loss')
plt.legend()

plt.show()

## LSTM Example

Now let's switch to more practical concerns: we will set up a model using a LSTM layer and train it on the IMDB data. Here's the network, similar to the one with SimpleRNN that we just presented. 

** Consider the network you have implemented above and add an `LSTM` layer with output dimension 32.**

** Compile the model with the same optimizer and same loss.**

** Fit the model as before.**

** Fill in the #TO DOs to complete the plotting.**

In [ ]:
acc = #TO DO: training accuracy
val_acc = #TO DO: validation accuracy
loss = #TO DO: training loss
val_loss = #TO DO: validation loss

# Accuracy
epochs = range(len(acc))
plt.plot(epochs, #TO DO, 'bo', label='Training acc') #TO DO: training accuracy
plt.plot(epochs, #TO DO, 'b', label='Validation acc') #TO DO: validation accuracy
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Loss
plt.plot(epochs, #TO DO, 'bo', label='Training loss') #TO DO: training loss
plt.plot(epochs, #TO DO, 'b', label='Validation loss') #TO DO: validation loss
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Using Word Embedding

In this second part of the Notebook, we will focus on Word Embedding, which is a powerful way to associate a vector with a word. "Word Embeddings" are low-dimensional floating point vectors (i.e. "dense" vectors, as opposed to sparse vectors). It is common to see word embeddings that are 256-dimensional, 512-dimensional, or 1024-dimensional when dealing with very large vocabularies.

There are two ways to obtain word embeddings:
- Learn word embeddings jointly with the main task you care about (e.g. document classification or sentiment prediction). In this setup, you would start with random word vectors, then learn your word vectors in the same way that you learn the weights of a neural network.
- Load into your model word embeddings that were pre-computed using a different machine learning task than the one you are trying to solve. These are called "pre-trained word embeddings".

### Learning Word Embedding with the `Embedding` layer

The simplest way to associate a dense vector to a word would be to pick the vector at random. The problem with this approach is that the resulting embedding space would have no structure: for instance, the words "accurate" and "exact" may end up with completely different embeddings, even though they are interchangeable in most sentences. It would be very difficult for a deep neural network to make sense of such a noisy, unstructured embedding space.

In real-world word embedding spaces, common examples of meaningful geometric transformations are "gender vectors" and "plural vector". For instance, by adding a "female vector" to the vector "king", one obtain the vector "queen". By adding a "plural vector", one obtain "kings". Word embedding spaces typically feature thousands of such interpretable and potentially useful vectors.

But more pragmatically, what makes a good word embedding space depends heavily on your task: the perfect word embedding space for an English-language movie review sentiment analysis model may look very different from the perfect embedding space for an English-language legal document classification model, because the importance of certain semantic relationships varies from task to task.

It is thus reasonable to learn a new embedding space with every new task. Thankfully, backpropagation makes this really easy, and Keras makes it even easier. It's just about learning the weights of a layer: the Embedding layer.

In [ ]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)

The Embedding layer is best understood as a dictionary mapping integer indices (which stand for specific words) to dense vectors. It takes as input integers, it looks up these integers into an internal dictionary, and it returns the associated vectors. It's effectively a dictionary lookup.

The Embedding layer takes as input a 2D tensor of integers, of shape (samples, sequence_length), where each entry is a sequence of integers. It can embed sequences of variable lengths, so for instance we could feed into our embedding layer above batches that could have shapes (32, 10) (batch of 32 sequences of length 10) or (64, 15) (batch of 64 sequences of length 15). All sequences in a batch must have the same length, though (since we need to pack them into a single tensor), so sequences that are shorter than others should be padded with zeros, and sequences that are longer should be truncated.

This layer returns a 3D floating point tensor, of shape (samples, sequence_length, embedding_dimensionality). Such a 3D tensor can then be processed by a RNN layer or a 1D convolution layer (in the next sections).


Let's apply this idea to the IMDB movie review sentiment prediction task that you are already familiar with. Let's quickly prepare the data. We will restrict the movie reviews to the top 10,000 most common words, and cut the reviews after only 20 words. Our network will simply learn 8-dimensional embeddings for each of the 10,000 words, turn the input integer sequences (2D integer tensor) into embedded sequences (3D float tensor), flatten the tensor to 2D, and train a single Dense layer on top for classification.

### Data Preprocessing

In [ ]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

### Building the Network

Let's build the following network:
- `Embedding` layer with input dimension 10000, output dimension 8 and input_length equal to maxlen. We specify the maximum input layer to the Embedding layer so we can later flatten the embedded inputs;
- `Flatten` layer: to flatten the 3D tensor of embeddings into a 2D tensor of shape (samples, maxlen * 8);
- Fully Connected layer, with `sigmoid` activation function;

**Build the network as described above. Check Keras documentation to see what is exactly input_length.**

** Get a summary of the model.**

** Compile the model with an optimizer and a loss.**

** Fit the model, as previously. **

** What do you think of your results? Draw some considerations.**

## Sequence Processing with Convnets

### Implementing a 1D convnet 

In Keras, you would use a 1D convnet via the Conv1D layer, which has a very similar interface to Conv2D. It takes as input 3D tensors with shape (samples, time, features) and also returns similarly-shaped 3D tensors. The convolution window is a 1D window on the temporal axis, axis 1 in the input tensor.


** What do you think can be the advantage of using this kind of approach with respect to classical RNN? Think a little bit about it and try to see which results you get from the implementation.**

Let's build a simple 2-layer 1D convnet and apply it to the IMDB sentiment classification task that you are already familiar with.

As a reminder, this is the code for obtaining and preprocessing the data:

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
max_len = 500  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

This is our example 1D convnet for the IMDB dataset. The network should look like this:
- `Embedding` layer, with input dimension equal to max_features, output dimension equal to 128 and input_length equal to max_len;
- `Conv1D` layer with 32 filters, kernel size 7 and ReLU activation function;
- `MaxPooling` layer with pooling size 5;
- `Conv1D` layer with 32 filters, kernel size 7 and ReLU activation function;
- `GlobalMaxPooling1D` layer;
- Output Fully Connected layer;

** Get a summary of the model.**

** Compile the model with an optimizer with some learning rate, and a loss function.**

** Fit the model, as before.**

In [ ]:
import matplotlib.pyplot as plt

acc = # TO DO
val_acc = # TO
loss = #TO DO
val_loss = # TO DO

epochs = range(len(acc))

plt.plot(epochs, #TO DO, 'bo', label='Training acc')
plt.plot(epochs, #TO DO, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, # TO DO, 'bo', label='Training loss')
plt.plot(epochs, #TO DO, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()